In [1]:
pip install xlsxwriter --user

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Python311\python.exe -m pip install --upgrade pip


In [ ]:
import xlsxwriter
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from keras.models import load_model
from openpyxl import load_workbook
import tensorflow_hub as hub
#Importing Universal sentence encoder
tf_hub_embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
                                        name="universal_sentence_encoder")

In [4]:
# Creating Model
def IPC_Self_learning_Model(text):
  train_data=pd.read_excel("ipc_data_1.xlsx")
  X = train_data["Offence"].to_list()
  X = np.array(X)
  train_data_1= train_data['Section']
  train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))

  multilabel = MultiLabelBinarizer()
  y = multilabel.fit_transform(train_data_1)


  # Define feature extractor model using TF Hub layer
  inputs = layers.Input(shape=[], dtype=tf.string)
  pretrained_embedding = tf_hub_embedding_layer(inputs) # tokenize text and create embedding
  x = layers.Dense(128, activation="relu")(pretrained_embedding) # add a fully connected layer on top of the embedding
  # Note: you could add more layers here if you wanted to
  outputs = layers.Dense(len(multilabel.classes_), activation="softmax")(x) # create the output layer
  model_1 = tf.keras.Model(inputs=inputs,
                          outputs=outputs)
  


  # Compile the model
  model_1.compile(loss="categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=["accuracy"])

  model_1.fit(X,  y,
               epochs=30,
                verbose=0)
  
  text = np.array([text])
  model_1_pred_probs = model_1.predict(text)
  n=2
  indices_2 = (-model_1_pred_probs).argsort()[:n]
  q=[]
  for r in range(0,4):
    q.append(multilabel.classes_[indices_2[0,r]])
  

  return q

In [5]:
offence_ ="Murder" #Enter any offence u want

In [6]:
pred_ipc = IPC_Self_learning_Model(offence_)

1/1 [==============================] - 0s 363ms/step


In [7]:
train_data=pd.read_excel("ipc_data_1.xlsx")
Off = train_data["Offence"]
punish = train_data["Punishment"]
train_data_1= train_data['Section']
train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))
Sec_1= train_data_1

data = {}  # Initialize data as an empty dictionary

for x in range(0, 382):
    data[Sec_1[x][0]] = [Off[x], punish[x]]

def data_retrival(code):
    detail = []
    punishment = []
    for i in code:  
        detail.append(data[i][0])
        punishment.append(data[i][1])
    for j in range(len(code)):
        print(f"{code[j]}\n Detail:{detail[j]}\n Punishment:{punishment[j]}")


In [8]:
data_retrival(pred_ipc)

302 IPC
 Detail:Murder
 Punishment:Death, or imprisonmentfor life, and fine
396 IPC
 Detail:Murder in dacoity
 Punishment:Death, imprisonment for life, or rigorous imprisonment for 10 years and fine
307 IPC
 Detail:Attempt to murder
 Punishment:Imprisonment for life, or imprisonment for 10 years and fine.
308 IPC
 Detail:Attempt to commit culpable homicide.
 Punishment:Imprisonment for 3 years, or fine, or both.


In [13]:
import tkinter as tk
from tkinter import ttk

# Function to handle button click event
def show_result():
    offence = offence_entry.get()
    pred_ipc = IPC_Self_learning_Model(offence)
    data_retrival(pred_ipc)

# Create tkinter window
root = tk.Tk()
root.title("IPC Offence Predictor")
root.geometry("400x300")  # Set window dimensions

# Style for the tkinter elements
style = ttk.Style()
style.configure("TButton", padding=10, font=("Arial", 12))
style.configure("TLabel", padding=10, font=("Arial", 12))

# Label and entry for entering offence
offence_label = ttk.Label(root, text="Enter Offence:")
offence_label.pack(pady=10)  # Add padding around the label
offence_entry = ttk.Entry(root, font=("Arial", 12))
offence_entry.pack(pady=10)  # Add padding around the entry field

# Button to predict IPC sections
predict_button = ttk.Button(root, text="Predict IPC Sections", command=show_result)
predict_button.pack(pady=20)  # Add padding around the button

# Frame to display results
result_frame = ttk.Frame(root)
result_frame.pack(padx=20, pady=10)  # Add padding around the frame
result_label = ttk.Label(result_frame, wraplength=350, font=("Arial", 11))
result_label.pack()

# Function to display results
def data_retrival(code):
    result_label.config(text="")
    detail = []
    punishment = []
    for i in code:
        detail.append(data[i][0])
        punishment.append(data[i][1])
    for j in range(len(code)):
        result_label.config(text=result_label.cget("text") + f"{code[j]}\n Detail: {detail[j]}\n Punishment: {punishment[j]}\n\n")

# Run the tkinter main loop
root.mainloop()


In [14]:
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras import layers
from keras.models import load_model
from openpyxl import load_workbook
import tensorflow_hub as hub
import tkinter as tk
from tkinter import ttk

# Importing Universal sentence encoder
tf_hub_embedding_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        trainable=False,
                                        name="universal_sentence_encoder")

def IPC_Self_learning_Model(text):
    train_data = pd.read_excel("ipc_data_1.xlsx")
    X = train_data["Offence"].to_list()
    X = np.array(X)
    train_data_1 = train_data['Section']
    train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))

    multilabel = MultiLabelBinarizer()
    y = multilabel.fit_transform(train_data_1)

    # Define feature extractor model using TF Hub layer
    inputs = layers.Input(shape=[], dtype=tf.string)
    pretrained_embedding = tf_hub_embedding_layer(inputs)
    x = layers.Dense(128, activation="relu")(pretrained_embedding)
    outputs = layers.Dense(len(multilabel.classes_), activation="softmax")(x)
    model_1 = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compile the model
    model_1.compile(loss="categorical_crossentropy",
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=["accuracy"])

    # Fit the model and store the training history
    history = model_1.fit(X, y, epochs=30, verbose=0)

    # Print the accuracy
    accuracy = history.history['accuracy'][-1]
    print(f"Final Training Accuracy: {accuracy}")

    text = np.array([text])
    model_1_pred_probs = model_1.predict(text)
    n = 2
    indices_2 = (-model_1_pred_probs).argsort()[:n]
    q = []
    for r in range(0, 4):
        q.append(multilabel.classes_[indices_2[0, r]])

    return q

def data_retrival(code):
    result_text.config(state=tk.NORMAL)
    result_text.delete(1.0, tk.END)
    detail = []
    punishment = []
    for i in code:
        detail.append(data[i][0])
        punishment.append(data[i][1])
    for j in range(len(code)):
        result_text.insert(tk.END, f"{code[j]}\n Detail: {detail[j]}\n Punishment: {punishment[j]}\n\n")
    result_text.config(state=tk.DISABLED)

# Load data
train_data = pd.read_excel("ipc_data_1.xlsx")
Off = train_data["Offence"]
punish = train_data["Punishment"]
train_data_1 = train_data['Section']
train_data_1 = train_data_1.apply(lambda x: ast.literal_eval(x))
Sec_1 = train_data_1

# Initialize data as an empty dictionary
data = {}
for x in range(0, 382):
    data[Sec_1[x][0]] = [Off[x], punish[x]]

# Create tkinter window
root = tk.Tk()
root.title("IPC Offence Predictor")
root.geometry("400x400")  # Set window dimensions

# Style for the tkinter elements
style = ttk.Style()
style.configure("TButton", padding=10, font=("Arial", 12))
style.configure("TLabel", padding=10, font=("Arial", 12))

# Label and entry for entering offence
offence_label = ttk.Label(root, text="Enter Offence:")
offence_label.pack(pady=10)
offence_entry = ttk.Entry(root, font=("Arial", 12))
offence_entry.pack(pady=10)

# Button to predict IPC sections
predict_button = ttk.Button(root, text="Predict IPC Sections", command=lambda: show_result())
predict_button.pack(pady=20)

# Frame to display results using Text widget
result_frame = ttk.Frame(root)
result_frame.pack(padx=20, pady=10)

result_text = tk.Text(result_frame, wrap=tk.WORD, height=10, width=40, font=("Arial", 11))
result_text.pack()

def show_result():
    offence = offence_entry.get()
    pred_ipc = IPC_Self_learning_Model(offence)
    data_retrival(pred_ipc)

# Run the tkinter main loop
root.mainloop()


Final Training Accuracy: 0.9817232489585876
1/1 [==============================] - 0s 297ms/step
